<a href="https://colab.research.google.com/github/bob8dod/NLP_project-Review_Analysis/blob/main/Crawling/Crawling_11st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 크롤링(11번가)

In [72]:
import json
import re
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool, Manager


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
products = 2777504082

url_basic = 'https://www.11st.co.kr/products/{}'.format(products)
data = requests.get(url_basic, headers=headers)
soup = BeautifulSoup(data.text, 'html.parser')

category_path = soup.find('div', attrs={'class':'c_product_category_path'}).find_all('em', attrs={'class':'selected'})
categories = set()
for cate in category_path:
    categories.add(cate.text)
product_name = soup.find('h1', attrs={'class':'title'}).text.strip()
img_src = soup.find('div', attrs={'class':'img_full'}).find('img')['src']
price = soup.find('ul', attrs={'class':'price_wrap'}).find('span',attrs={'class':'value'}).text
review_len = soup.find('strong', attrs={'class':'text_num'}).text
review_len = int(re.sub('[^0-9]','', review_len))
print('상품명:', product_name)
print('카테고리:', categories)
print('가격:', price)
print('모든 리뷰 개수:', review_len)

manager = Manager()
text = manager.list()
score = manager.list()

def Crawling(pageNo):
    try:
        url = 'https://m.11st.co.kr/products/v1/app/products/{}/reviews/list?pageNo={}&sortType=01&pntVals=&rtype=&themeNm='.format(products, pageNo)
        response = urlopen(url)
        json_data = json.load(response)
        for i in range(len(json_data['review']['list'])):
            if json_data['review']['list'][i]['subject']:
                review = json_data['review']['list'][i]['subject']
                score = str(json_data['review']['list'][i]['evlPnt'])
                text.append([score, review])
    except:
        print('리뷰가 끝났습니다. 총 개수:', len(text))

if __name__ == '__main__':
    start_time = time.time()
    pool = Pool(processes=32)
    pool.map(Crawling,range(review_len//10 + 1))
    pool.close()
    pool.join()
    print('수집한 리뷰 총 개수: ', len(text)) # 빈 리뷰 제외 한 총 리뷰 개수
    print("실행 시간 : %s초" % (time.time() - start_time))

상품명: 프레시지 블랙라벨 스테이크 (2인분) 밀키트 쿠킹박스
카테고리: {'식품'}
가격: 15,680
모든 리뷰 개수: 9690
수집한 리뷰 총 개수:  7151
실행 시간 : 36.577216148376465초


## New Text2Csv


In [78]:
import pandas as pd
data = pd.DataFrame(list(text), columns=['score','review'])
print('리뷰 데이터 총 개수:', len(data))
data.drop_duplicates(['review'], inplace=True)
data.reset_index(drop=True, inplace=True)
print('중복제거 후:', len(data))
data.head(3)

리뷰 데이터 총 개수: 7151
중복제거 후: 6731


,score,review
0,5,좋은 상품을 저렴하게 구입할 수 있어서 좋아요
1,5,채소가 약간 시들했지만 늘 구매하는 제품이요
2,5,딜 뜰때마다 구매하는 품목입니다. 맛있게 잘 먹었습니다.


In [88]:
#csv로 저장 -> {이름} 리뷰({리뷰개수}개)
data.to_csv('{} 리뷰({}개).csv'.format(product_name,len(data)), index=False )